In [1]:
%run ~/relmapping/annot/notebooks/__init__.ipynb

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/b2/scratch/ahringer/jj374/lab/relmapping


In [2]:
# Load hypersensitive sites
df_sites = pd.read_csv('annot/S1_accessible_sites/S1a_accessible_sites.tsv', sep='\t')[['chrom', 'start', 'end', 'atac_mode']]
#df_sites = df_sites.head(100)
print(len(df_sites), 'ATAC-seq sites')

42245 ATAC-seq sites


In [3]:
# Read lcap
def read_startbp_(chroms, modes, stage, strand, inner_flank=50, outer_flank=300):
    step_fwd = 'trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd'
    step_rev = 'trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev'
    if strand == '+':
        fp_rep1 = pf('lcap808_%(stage)s_rep1' % locals(), step_fwd, '.bw', 'lcap808')
        fp_rep2 = pf('lcap808_%(stage)s_rep2' % locals(), step_fwd, '.bw', 'lcap808')
    else:
        fp_rep1 = pf('lcap808_%(stage)s_rep1' % locals(), step_rev, '.bw', 'lcap808')
        fp_rep2 = pf('lcap808_%(stage)s_rep2' % locals(), step_rev, '.bw', 'lcap808')

    (r_starts, r_ends) = ((modes - outer_flank).tolist(), (modes - inner_flank).tolist())    
    (f_starts, f_ends) = ((modes + inner_flank).tolist(), (modes + outer_flank).tolist())
    if strand == '+':
        (n_starts, n_ends) = (r_starts, r_ends)
        (p_starts, p_ends) = (f_starts, f_ends)
    else:
        (n_starts, n_ends) = (f_starts, f_ends)
        (p_starts, p_ends) = (r_starts, r_ends)

    def f_(s): return int(np.nansum(s))

    df = pd.DataFrame()
    df['rep1_ncount' % locals()] = list(map(f_, yp.mread_regions(fp_rep1, chroms, n_starts, n_ends)))
    df['rep2_ncount' % locals()] = list(map(f_, yp.mread_regions(fp_rep2, chroms, n_starts, n_ends))) 
    df['rep1_pcount' % locals()] = list(map(f_, yp.mread_regions(fp_rep1, chroms, p_starts, p_ends)))
    df['rep2_pcount' % locals()] = list(map(f_, yp.mread_regions(fp_rep2, chroms, p_starts, p_ends))) 
    return df

def read_median_(chroms, modes, stage, strand, inner_flank=50, outer_flank=300):
    step_fwd = 'trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd'
    step_rev = 'trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev'
    if strand == '+':
        fp_rep1 = pf('lcap808_%(stage)s_rep1' % locals(), step_fwd, '.bw', 'lcap808')
        fp_rep2 = pf('lcap808_%(stage)s_rep2' % locals(), step_fwd, '.bw', 'lcap808')
    else:
        fp_rep1 = pf('lcap808_%(stage)s_rep1' % locals(), step_rev, '.bw', 'lcap808')
        fp_rep2 = pf('lcap808_%(stage)s_rep2' % locals(), step_rev, '.bw', 'lcap808')

    # median with exclusion zone
    (r_starts, r_ends) = ((modes - outer_flank).tolist(), (modes - inner_flank).tolist())    
    (f_starts, f_ends) = ((modes + inner_flank).tolist(), (modes + outer_flank).tolist())

    if strand == '+':
        (n_starts, n_ends) = (r_starts, r_ends)
        (p_starts, p_ends) = (f_starts, f_ends)
    else:
        (n_starts, n_ends) = (f_starts, f_ends)
        (p_starts, p_ends) = (r_starts, r_ends)

    def f_(a): return int(np.median(np.nan_to_num(a)))

    df = pd.DataFrame()
    df['rep1_ncount' % locals()] = list(map(f_, yp.mread_regions(fp_rep1, chroms, n_starts, n_ends)))
    df['rep2_ncount' % locals()] = list(map(f_, yp.mread_regions(fp_rep2, chroms, n_starts, n_ends))) 
    df['rep1_pcount' % locals()] = list(map(f_, yp.mread_regions(fp_rep1, chroms, p_starts, p_ends)))
    df['rep2_pcount' % locals()] = list(map(f_, yp.mread_regions(fp_rep2, chroms, p_starts, p_ends))) 
    return df

def read_merge_(df1, df2):
    def select_(s1, s2):
        fc_s1 = math.log(s1[['rep1_pcount', 'rep2_pcount']].sum() + 1, 2)\
              - math.log(s1[['rep1_ncount', 'rep2_ncount']].sum() + 1, 2)
        fc_s2 = math.log(s2[['rep1_pcount', 'rep2_pcount']].sum() + 1, 2)\
              - math.log(s2[['rep1_ncount', 'rep2_ncount']].sum() + 1, 2)
        if fc_s2 > fc_s1:
            return s2
        else:
            return s1
    return pd.DataFrame.from_records(map(lambda ir1, ir2: select_(ir1[1], ir2[1]), df1.iterrows(), df2.iterrows()))

In [ ]:
def lcap_count(chroms, modes, stage):
    !date
    print(stage)
    # Naming always relative to the forward strand
    df_ = pd.DataFrame()
    df_[['lcap_%(stage)s_rep1_fwd_rcount' % locals(), 'lcap_%(stage)s_rep2_fwd_rcount' % locals(),
         'lcap_%(stage)s_rep1_fwd_fcount' % locals(), 'lcap_%(stage)s_rep2_fwd_fcount' % locals(),]] = \
            read_merge_(
                read_startbp_(chroms, modes, stage, '+'), 
                read_median_(chroms, modes, stage, '+')
            )

    df_[['lcap_%(stage)s_rep1_rev_fcount' % locals(), 'lcap_%(stage)s_rep2_rev_fcount' % locals(),
         'lcap_%(stage)s_rep1_rev_rcount' % locals(), 'lcap_%(stage)s_rep2_rev_rcount' % locals(),]] = \
            read_merge_(
                read_startbp_(chroms, modes, stage, '-'),
                read_median_(chroms, modes, stage, '-'),
            )

    return df_

df_ = pd.concat([lcap_count(df_sites['chrom'], df_sites['atac_mode'], stage)
                 for stage in itertools.islice(config['stages'], None)], axis=1)
df_counts = pd.concat([df_sites[['chrom', 'start', 'end', 'atac_mode']], df_], axis=1)
df_counts.head()

Tue 13 Mar 16:20:28 GMT 2018
wt_emb
mread_regions False lcap808_wt_emb_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_39d6b775611abec17787c5856f86815d.tmp
mread_regions False lcap808_wt_emb_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_532fe65539b9c4fbc6953634801cbe2e.tmp
mread_regions False lcap808_wt_emb_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_6b55a327ae8882a3f83275d687df623e.tmp
mread_regions False lcap808_wt_emb_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_14e4b23b05159f771f5f80325bf2fd0c.tmp
mread_regions True lcap808_wt_emb_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_39d6b775611abec17787c5856f86815d.tmp
mread_regions True lcap808_wt_emb_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_r

mread_regions True lcap808_wt_l2_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_276607896cad07a1c745c7efd9f05add.tmp
mread_regions True lcap808_wt_l2_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_7b3b1af8576d08b1fbc9da263e1ac5bb.tmp
Tue 13 Mar 17:22:18 GMT 2018
wt_l3
mread_regions False lcap808_wt_l3_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_2e013f558f52a2492e6006545445e504.tmp
mread_regions False lcap808_wt_l3_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_84bcef5de449b7df188167f7c3bd236b.tmp
mread_regions False lcap808_wt_l3_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_fa478e3a35259eabf2608467311bdebb.tmp
mread_regions False lcap808_wt_l3_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_bro

mread_regions True lcap808_wt_ya_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_5ee9ddea8888c9d9c4efe9649cd1911b.tmp
mread_regions True lcap808_wt_ya_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_fd17fe81aac4f1930e3ceb2e76fda934.tmp
mread_regions True lcap808_wt_ya_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_93d3678bff6262c3316b1c11a0bbabdd.tmp
Tue 13 Mar 18:26:31 GMT 2018
glp1_d1
mread_regions False lcap808_glp1_d1_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_61444291642dde6a90b57e1d7551a991.tmp
mread_regions False lcap808_glp1_d1_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_dbf0db1dbf9d9ef269cca667cec216d7.tmp
mread_regions False lcap808_glp1_d1_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_r

mread_regions False lcap808_glp1_d6_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_df7c126ea36d4550013bebac67a17ca1.tmp
mread_regions True lcap808_glp1_d6_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_40ad858fc8f53ce623eb8d4744d73e5e.tmp
mread_regions True lcap808_glp1_d6_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_49604f69bd0b8cb4d5b303d9b1981222.tmp
mread_regions True lcap808_glp1_d6_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_9626c6cf4c1968241f0a5cbd6b0cd1d7.tmp
mread_regions True lcap808_glp1_d6_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_df7c126ea36d4550013bebac67a17ca1.tmp
Tue 13 Mar 19:31:10 GMT 2018
glp1_d9
mread_regions False lcap808_glp1_d9_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM

,chrom,start,end,atac_mode,lcap_wt_emb_rep1_fwd_rcount,lcap_wt_emb_rep2_fwd_rcount,lcap_wt_emb_rep1_fwd_fcount,lcap_wt_emb_rep2_fwd_fcount,lcap_wt_emb_rep1_rev_fcount,lcap_wt_emb_rep2_rev_fcount,...,lcap_glp1_d9_rep1_rev_rcount,lcap_glp1_d9_rep2_rev_rcount,lcap_glp1_d13_rep1_fwd_rcount,lcap_glp1_d13_rep2_fwd_rcount,lcap_glp1_d13_rep1_fwd_fcount,lcap_glp1_d13_rep2_fwd_fcount,lcap_glp1_d13_rep1_rev_fcount,lcap_glp1_d13_rep2_rev_fcount,lcap_glp1_d13_rep1_rev_rcount,lcap_glp1_d13_rep2_rev_rcount
0,chrI,1870,2092,1981,0,0,0,0,0,0,...,23,19,0,0,0,0,0,0,0,0
1,chrI,3809,3994,3901,0,0,0,0,1,0,...,515,640,0,1,3,3,0,0,445,317
2,chrI,4302,4410,4356,0,0,0,0,3,4,...,5,10,0,0,0,0,2,2,5,2
3,chrI,11245,11465,11355,8,1,14,8,0,0,...,0,0,0,4,79,118,0,0,5,3
4,chrI,13087,13207,13147,0,0,0,0,0,0,...,1,7,94,87,89,97,1,1,1,2


In [ ]:
df_counts.to_csv('annot/S2_regulatory_annotation/_lcap_jump/lcap_counts.tsv', sep='\t', index=False)

In [ ]:
def read_incr(chroms, modes, stage, strand, inner_flank=50, outer_flank=300):
    step_fwd = 'trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd'
    step_rev = 'trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev'
    #step_fwd = 'coverage_q10_reads_by_rep_fwd'
    #step_rev = 'coverage_q10_reads_by_rep_rev'
    if strand == '+':
        fp_rep1 = pf('lcap808_%(stage)s_rep1' % locals(), step_fwd, '.bw', 'lcap808')
        fp_rep2 = pf('lcap808_%(stage)s_rep2' % locals(), step_fwd, '.bw', 'lcap808')
    else:
        fp_rep1 = pf('lcap808_%(stage)s_rep1' % locals(), step_rev, '.bw', 'lcap808')
        fp_rep2 = pf('lcap808_%(stage)s_rep2' % locals(), step_rev, '.bw', 'lcap808')

    if strand == '+':
        (n_starts, n_ends) = ((modes - outer_flank).tolist(), (modes - inner_flank).tolist())
        (p_starts, p_ends) = ((modes - inner_flank).tolist(), (modes + outer_flank).tolist())
    else:
        (n_starts, n_ends) = ((modes + inner_flank).tolist(), (modes + outer_flank).tolist())
        (p_starts, p_ends) = ((modes - outer_flank).tolist(), (modes + inner_flank).tolist())

    def f_(s): return int(np.nansum(s)) # int(yp.nanmax(s))

    df = pd.DataFrame()
    df['rep1_ncount' % locals()] = list(map(f_, yp.mread_regions(fp_rep1, chroms, n_starts, n_ends)))
    df['rep2_ncount' % locals()] = list(map(f_, yp.mread_regions(fp_rep2, chroms, n_starts, n_ends))) 
    df['rep1_pcount' % locals()] = list(map(f_, yp.mread_regions(fp_rep1, chroms, p_starts, p_ends)))
    df['rep2_pcount' % locals()] = list(map(f_, yp.mread_regions(fp_rep2, chroms, p_starts, p_ends))) 
    return df

def lcap_count_incr(chroms, modes, stage):
    !date
    print(stage)
    # Naming always relative to the forward strand
    df_ = pd.DataFrame()
    df_[['lcap_%(stage)s_rep1_fwd_rcount' % locals(), 'lcap_%(stage)s_rep2_fwd_rcount' % locals(),
         'lcap_%(stage)s_rep1_fwd_fcount' % locals(), 'lcap_%(stage)s_rep2_fwd_fcount' % locals(),]] = \
                                                                    read_incr(chroms, modes, stage, '+')

    df_[['lcap_%(stage)s_rep1_rev_fcount' % locals(), 'lcap_%(stage)s_rep2_rev_fcount' % locals(),
         'lcap_%(stage)s_rep1_rev_rcount' % locals(), 'lcap_%(stage)s_rep2_rev_rcount' % locals(),]] = \
                                                                    read_incr(chroms, modes, stage, '-')
    return df_

df_ = pd.concat([lcap_count_incr(df_sites['chrom'], df_sites['atac_mode'], stage)
                 for stage in itertools.islice(config['stages'], None)], axis=1)
df_counts = pd.concat([df_sites[['chrom', 'start', 'end', 'atac_mode']], df_], axis=1)

Tue 13 Mar 20:13:11 GMT 2018
wt_emb
mread_regions True lcap808_wt_emb_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_39d6b775611abec17787c5856f86815d.tmp
mread_regions True lcap808_wt_emb_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_532fe65539b9c4fbc6953634801cbe2e.tmp
mread_regions False lcap808_wt_emb_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_ce1b95e73667e286695d22c5a275261f.tmp
mread_regions False lcap808_wt_emb_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_eef0e352689246bc0c7445f026fac5b3.tmp
mread_regions True lcap808_wt_emb_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_7134f28eebe39ff8c800e68c3d2e0563.tmp
mread_regions True lcap808_wt_emb_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRN

mread_regions False lcap808_wt_ya_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_6a958ad897611c8f176d7a3da8e3a890.tmp
mread_regions False lcap808_wt_ya_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_rev.bw.mread_regions_e70e72639a9aabc7b8ec7f8b67bacf65.tmp
Tue 13 Mar 21:09:49 GMT 2018
glp1_d1
mread_regions True lcap808_glp1_d1_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_61444291642dde6a90b57e1d7551a991.tmp
mread_regions True lcap808_glp1_d1_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_dbf0db1dbf9d9ef269cca667cec216d7.tmp
mread_regions False lcap808_glp1_d1_rep1.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.startbp_fwd.bw.mread_regions_ea959bc1ef4254732053bbbb6d56ef63.tmp
mread_regions False lcap808_glp1_d1_rep2.trim20.bwa_pe.rm_unmapped_pe.rm_chrM.r

In [ ]:
df_counts.to_csv('annot/S2_regulatory_annotation/_lcap_jump/lcap_counts_incr.tsv', sep='\t', index=False)